Телекоммуникационный сектор в Индии быстро меняется: создается все больше и больше телекоммуникационных предприятий, и многие клиенты решают переключаться между поставщиками услуг. «Отток» относится к процессу, когда клиенты или подписчики перестают пользоваться услугами или продуктами компании. Понимание факторов, которые влияют на удержание клиента в качестве клиента при прогнозировании оттока, имеет решающее значение для телекоммуникационных компаний для повышения качества обслуживания и удовлетворенности клиентов. Этот проект направлен на изучение сложной динамики поведения и демографии клиентов в индийском телекоммуникационном секторе для прогнозирования оттока клиентов с использованием двух комплексных наборов данных от четырех основных телекоммуникационных партнеров: Airtel, Reliance Jio, Vodafone и BSNL:

- `telecom_demographics.csv` содержит информацию, связанную с демографическими данными клиентов из Индии:

| Переменная             | Описание                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Уникальный идентификатор для каждого клиента.|
| `telecom_partner `     | Телекоммуникационный партнер, связанный с клиентом.|
| `gender `              | Пол клиента.                                    |
| `age `                 | Возраст клиента.                                |
| `state`                | Индийский штат, в котором находится клиент.|
| `city`                 | Город, в котором находится клиент.       |
| `pincode`              | Пин-код местоположения клиента.          |
| `registration_event` | Когда клиент зарегистрировался у телекоммуникационного партнера.|
| `num_dependents`      | Количество иждивенцев (например, детей) у клиента.|
| `estimated_salary`     | Ориентировочная зарплата клиента.                 |

- `telecom_usage` содержит информацию о моделях использования индийских клиентов:

| Переменная   | Описание                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Уникальный идентификатор для каждого клиента.               |
| `calls_made` | Количество звонков, совершенных клиентом.                    |
| `sms_sent`   | Количество SMS-сообщений, отправленных клиентом.             |
| `data_used`  | Объем данных, используемых клиентом.                         |
| `churn`    | Бинарная переменная, указывающая, есть отток или нет (1 = отток, 0 = не отток).|


In [3]:
# Импорт необходимых библиотек и методов/функций
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Загрузка данных
telco_demog = pd.read_csv('telecom_demographics.csv')
telco_usage = pd.read_csv('telecom_usage.csv')

# Объединение данных
churn_df = telco_demog.merge(telco_usage, on='customer_id')

# Определение уровня оттока
churn_rate = churn_df['churn'].value_counts() / len(churn_df)
print(churn_rate)

# Идентификация категориальных переменных
print(churn_df.info())

# Быстрое кодирование категориальных переменных
churn_df = pd.get_dummies(churn_df, columns=['telecom_partner', 'gender', 'state', 'city', 'registration_event'])

# Особенности масштабирования
scaler = StandardScaler()

# Удаление «customer_id», поскольку это не функция
features = churn_df.drop(['customer_id', 'churn'], axis=1)
features_scaled = scaler.fit_transform(features)

# Определение целевой переменной
target = churn_df['churn']

# Разделение набора данных
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Создание логистической регрессии
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

# Прогнозирование с Logistic Regression
logreg_pred = logreg.predict(X_test)

# Оценка Logistic Regression
print(confusion_matrix(y_test, logreg_pred))
print(classification_report(y_test, logreg_pred))

# Создание модели Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Прогнозирование с Random Forest
rf_pred = rf.predict(X_test)

# Оценка Random Forest
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

0    0.799538
1    0.200462
Name: churn, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6500 entries, 0 to 6499
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         6500 non-null   int64 
 1   telecom_partner     6500 non-null   object
 2   gender              6500 non-null   object
 3   age                 6500 non-null   int64 
 4   state               6500 non-null   object
 5   city                6500 non-null   object
 6   pincode             6500 non-null   int64 
 7   registration_event  6500 non-null   object
 8   num_dependents      6500 non-null   int64 
 9   estimated_salary    6500 non-null   int64 
 10  calls_made          6500 non-null   int64 
 11  sms_sent            6500 non-null   int64 
 12  data_used           6500 non-null   int64 
 13  churn               6500 non-null   int64 
dtypes: int64(9), object(5)
memory usage: 761.7+ KB
None
[[920 107]
 